In [1]:
# Numerical and plotting packages
import seaborn as sns
import numpy as np
import pandas as pd
import statsmodels.api as sm
import natsort
from scipy.signal import sosfiltfilt, butter, hilbert
# Libraries needed for this notebook to interact with the DANDI API
from pynwb import NWBHDF5IO
from dandi.dandiapi import DandiAPIClient

# Libraries needed for this notebook to interact with NWB events
from ndx_events import LabeledEvents, AnnotatedEventsTable, Events

# FSSpec is a library that allows us to read files from the cloud
import fsspec

# NWB is based on HF5, so we need this library to read NWB files
import h5py
from fsspec.implementations.cached import CachingFileSystem

# Getting a list of Path Locations

In [2]:
with DandiAPIClient() as client:
    paths = []
    for file in client.get_dandiset("000055", "draft").get_assets_with_path_prefix(""):
        paths.append(file.path)
paths = natsort.natsorted(paths)
# print(paths)

## Access to data on cloud

In [3]:
sbj, session = 1, 3  # participant 1, session 3
path = f'sub-0{sbj}/sub-0{sbj}_ses-{session}_behavior+ecephys.nwb'
# behavior_type = 'Eat' # only analyze data during eating
# neural_freq_range = [80, 100]  # Frequency band of interest in Hz
# ecog_ch_num = 7 # electrode number over motor cortex
# keypoint_of_interest = 'R_Wrist' # right wrist movement
# pose_direction = 'vertical'  # 'vertical' or 'horizontal'

In [4]:
def get_nwb(sbj, session):
    """ return nwbfile

    Parameters
    ----------
    sbj : _type_
        _description_
    session : _type_
        _description_

    Returns
    -------
    _type_
        _description_
    """

    path = f'sub-0{sbj}/sub-0{sbj}_ses-{session}_behavior+ecephys.nwb'
    with DandiAPIClient() as client:
        asset = client.get_dandiset("000055").get_asset_by_path(
            path=path)
        s3_path = asset.get_content_url(follow_redirects=1, strip_query=True)
        

    # Note, caching is set once per access. If you want to change the cache location, you will need to restart the kernel.
    fs = CachingFileSystem(
        fs=fsspec.filesystem("http"),
        # cache_storage="nwb-cache",  # Local folder for the cache
    )

    f = fs.open(s3_path, "rb")
    file = h5py.File(f)
    io = NWBHDF5IO(file=file, mode='r', load_namespaces=True)
    nwbfile = io.read()

    return nwbfile, fs

In [5]:
sbj_1_session_3, fs = get_nwb(1, 3)

/Applications/anaconda3/envs/NMA/lib/python3.9/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.4.0-alpha because version 1.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/Applications/anaconda3/envs/NMA/lib/python3.9/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'core' version 2.2.5 because version 2.6.0-alpha is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/Applications/anaconda3/envs/NMA/lib/python3.9/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.1.0 because version 0.4.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [6]:
# from AJILE12.plot_utils import prune_clabels, plot_clabels
# # Count activity and blocklist coarse label durations for each participant
# from AJILE12.plot_utils import clabel_table_create
# blocklist_labels = False  # show blocklist (True) or activity (False) label durations

# if blocklist_labels:
#     common_acts = [
#         'Blocklist (Data break)',
#         'Blocklist (Camera move/zoom)',
#         'Blocklist (Camera occluded)',
#         'Blocklist (Experiment)',
#         'Blocklist (Private time)',
#         'Blocklist (Tether/bandage)',
#         'Blocklist (Hands under blanket)',
#         'Blocklist (Clinical procedure)',
#     ]
# else:
#     common_acts = [
#         'Sleep/rest',
#         'Inactive',
#         'Talk',
#         'TV',
#         'Computer/phone',
#         'Eat',
#         'Other activity',
#     ]

# # Generate table
# clabel_table_create(common_acts,fs=fs)

In [7]:
clabels_orig = sbj_1_session_3.intervals['epochs'].to_dataframe()

In [8]:
common_acts = [
        'Sleep/rest',
        'Inactive',
        'Talk',
        'TV',
        'Computer/phone',
        'Eat',
        'Other activity',
    ]

In [9]:
clabels_orig

,start_time,stop_time,labels
id,,,
0,0.000000,86.533333,Blocklist (Data break)
1,86.533333,206.200000,Sleep/rest
2,206.200000,206.400000,Blocklist (Data break)
3,206.400000,326.166667,Sleep/rest
4,326.166667,326.333333,Blocklist (Data break)
...,...,...,...
1422,86135.433333,86135.600000,Blocklist (Data break)
1423,86135.600000,86255.500000,Sleep/rest
1424,86255.500000,86255.566667,Blocklist (Data break)


In [10]:
sleep_stamps = clabels_orig[clabels_orig['labels'] == 'Sleep/rest']

In [11]:
sbj_1_session_3

root pynwb.file.NWBFile at 0x6369795424
Fields:
  acquisition: {
    ECGL <class 'pynwb.base.TimeSeries'>,
    ECGR <class 'pynwb.base.TimeSeries'>,
    EOGL <class 'pynwb.base.TimeSeries'>,
    EOGR <class 'pynwb.base.TimeSeries'>,
    ElectricalSeries <class 'pynwb.ecephys.ElectricalSeries'>
  }
  devices: {
    ECG <class 'pynwb.device.Device'>,
    EOG <class 'pynwb.device.Device'>,
    GRID <class 'pynwb.device.Device'>,
    LAT <class 'pynwb.device.Device'>,
    LID <class 'pynwb.device.Device'>,
    LMT <class 'pynwb.device.Device'>,
    LPT <class 'pynwb.device.Device'>,
    LTO <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    ECG <class 'pynwb.ecephys.ElectrodeGroup'>,
    EOG <class 'pynwb.ecephys.ElectrodeGroup'>,
    GRID <class 'pynwb.ecephys.ElectrodeGroup'>,
    LAT <class 'pynwb.ecephys.ElectrodeGroup'>,
    LID <class 'pynwb.ecephys.ElectrodeGroup'>,
    LMT <class 'pynwb.ecephys.ElectrodeGroup'>,
    LPT <class 'pynwb.ecephys.ElectrodeGroup'>,
    LTO <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  epochs: epochs <class 'pynwb.epoch.TimeIntervals'>
  file_create_date: [datetime.datetime(2021, 6, 9, 5, 44, 48, 194751, tzinfo=tzoffset(None, -14400))]
  identifier: 4c571b6c-1028-476f-b0e1-e34aa27b3206
  intervals: {
    epochs <class 'pynwb.epoch.TimeIntervals'>,
    reaches <class 'pynwb.epoch.TimeIntervals'>
  }
  processing: {
    behavior <class 'pynwb.base.ProcessingModule'>
  }
  session_description: no description
  session_id: 3
  session_start_time: 2000-01-02 19:00:00-05:00
  subject: subject pynwb.file.Subject at 0x6369795136
Fields:
  species: Homo sapiens
  subject_id: 01

  timestamps_reference_time: 2000-01-02 19:00:00-05:00

In [12]:
sbj_1_session_3.acquisition['ElectricalSeries'].data.shape

(43200000, 94)

In [13]:
type(sbj_1_session_3)

pynwb.file.NWBFile

In [14]:
sbj_1_session_3.electrodes.to_dataframe()

,x,y,z,imp,location,filtering,group,group_name,standard_deviation,kurtosis,median_deviation,good,low_freq_R2,high_freq_R2
id,,,,,,,,,,,,,,
0,-45.648495,38.560372,36.187962,NaN,unknown,250 Hz lowpass,GRID pynwb.ecephys.ElectrodeGroup at 0x6369792...,GRID,35.398535,6.154781,28.978650,False,0.013106,-0.001107
1,-47.347152,31.182861,41.176799,NaN,unknown,250 Hz lowpass,GRID pynwb.ecephys.ElectrodeGroup at 0x6369792...,GRID,55.518727,1.309332,48.247142,True,-0.004729,-0.019041
2,-50.291409,22.157208,45.191885,NaN,unknown,250 Hz lowpass,GRID pynwb.ecephys.ElectrodeGroup at 0x6369792...,GRID,33.992008,1.585154,31.077800,True,-0.003226,-0.003235
3,-51.358961,11.787268,48.726509,NaN,unknown,250 Hz lowpass,GRID pynwb.ecephys.ElectrodeGroup at 0x6369792...,GRID,33.701261,2.652564,28.651509,True,0.036002,-0.013343
4,-51.219818,1.713889,53.000981,NaN,unknown,250 Hz lowpass,GRID pynwb.ecephys.ElectrodeGroup at 0x6369792...,GRID,59.648470,2.626867,51.301159,True,0.096380,-0.014212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,-34.106273,-88.149114,-19.819989,NaN,unknown,250 Hz lowpass,LTO pynwb.ecephys.ElectrodeGroup at 0x63697928...,LTO,58.315750,2.708134,42.559665,True,-0.009470,-0.007675
90,-44.753871,-80.476379,-19.968393,NaN,unknown,250 Hz lowpass,LTO pynwb.ecephys.ElectrodeGroup at 0x63697928...,LTO,55.756859,3.443116,40.267286,True,-0.008794,-0.011022
91,-51.026277,-73.221686,-18.328451,NaN,unknown,250 Hz lowpass,LTO pynwb.ecephys.ElectrodeGroup at 0x63697928...,LTO,48.797883,2.870422,35.445855,True,-0.002914,-0.013156


In [15]:

sbj_1_session_3.acquisition['ElectricalSeries'].data.shape

(43200000, 94)

In [16]:
type(sbj_1_session_3.acquisition['EOGL'].data)

h5py._hl.dataset.Dataset

In [17]:
sleep_stamps

,start_time,stop_time,labels
id,,,
1,86.533333,206.200000,Sleep/rest
3,206.400000,326.166667,Sleep/rest
5,326.333333,446.033333,Sleep/rest
7,446.166667,565.933333,Sleep/rest
9,566.033333,687.000000,Sleep/rest
...,...,...,...
1417,85775.900000,85895.633333,Sleep/rest
1419,85895.766667,86015.466667,Sleep/rest
1421,86015.666667,86135.433333,Sleep/rest


In [18]:
sleep_stamps['start_time'][3]/86400 * 43200000

103200.0

In [19]:
sleep_stamps.shape[0]

293

In [20]:
sleep_stamps['start_time'].index

Int64Index([   1,    3,    5,    7,    9,   11,   13,   15,   17,   19,
            ...
            1407, 1409, 1411, 1413, 1415, 1417, 1419, 1421, 1423, 1425],
           dtype='int64', name='id', length=293)

In [21]:
def extract_phys_accod_timestamps(phys_data:h5py._hl.dataset.Dataset, behavior_timestamps:pd.DataFrame, trange=list):

    """ Return a dictionary that contains EOG/ECG recordings according timestamps

    Args:
        phys_data: recordings from nwb files
        behavior_timestaps: a dataframe containing the timestamps labelled with specific activity
        trange: a list denoting the temporal region to look at, measured in hour

    """

    if len(trange) != 2:
        raise ValueError(f'trange should contains 2 elements instead of {len(trange)}')
    
    start_limit, end_limit = trange[0]*3600, trange[1]*3600 # convert to seconds
    len_timepoints = phys_data.shape[0] # get the number of recording timepoints
    phys_behavior = {}

    for i in sleep_stamps['start_time'].index:

        start_time, end_time = sleep_stamps['start_time'][i], sleep_stamps['stop_time'][i]
        start_idx, end_idx = (start_time/86400) * len_timepoints, (end_time/86400) * len_timepoints # convert seconds to corresponding index 
        start_idx, end_idx = int(start_idx), int(end_idx)

        if (start_time >= start_limit and start_time <= end_limit):
            if (end_time <= end_limit and end_time <= start_limit): # if within the range
                phys_behavior[str(start_time) + '_' + str(end_time)] = phys_data[start_idx:end_idx, :]
                

            elif (end_time >= end_limit and end_time <= start_limit):
                end_idx = (end_limit/86400) * len_timepoints
                end_idx = int(end_idx)
                phys_behavior[str(start_limit) + '_' + str(end_limit)] = phys_data[start_idx:end_idx, :]
                
                

        elif (start_time <= start_limit and start_time <= end_limit):
            if (end_time <= end_limit and end_time <= start_limit):
                start_idx = (start_limit/86400) * len_timepoints
                start_idx = int(start_idx)
                phys_behavior[str(start_limit) + '_' + str(end_limit)] = phys_data[start_idx:end_idx, :]
                
                

            elif (end_time>=start_limit and end_time >= end_limit):
                end_idx = (end_limit/86400) * len_timepoints
                end_idx = int(end_idx)
                start_idx = (start_limit/86400) * len_timepoints
                start_idx = int(start_idx)
                phys_behavior[str(start_limit) + '_' + str(end_limit)] = phys_data[start_idx:end_idx, :]
                
                
        else:
            pass
    
    return phys_behavior


In [22]:
extract_phys_accod_timestamps(sbj_1_session_3.acquisition['ElectricalSeries'].data, sleep_stamps, trange=[13, 17])

{'46800_61200': array([], shape=(0, 94), dtype=float32)}

In [23]:
a = int(46800/86400 * 43200000)
b = int(61200/86400 * 43200000)

In [24]:
sbj_1_session_3.acquisition['EOGL'].data[a:b]

array([21.67391 , 18.054855, 16.860912, ..., 14.500227, 18.531193,
       20.075861], dtype=float32)

In [25]:
sbj_1_session_3.acquisition['EOGL'].data[0:20]

array([-2.0812898 , -3.239176  , -0.60245   , -1.4832982 , -2.1988645 ,
       -1.6928512 , -3.106294  , -0.83714765, -3.5523083 , -8.957111  ,
       -1.3257879 , -2.418092  , -9.688462  , -7.673757  , -7.643367  ,
       -6.733464  , -4.7467494 , -9.410007  , -4.8335013 ,  1.4372702 ],
      dtype=float32)

In [26]:
import mne

In [32]:
import pandas as pd
import numpy as np
from mne.channels import make_dig_montage
from mne import create_info
from mne.io import RawArray

# Assuming sbj_1_session_3.electrodes.to_dataframe() returns a DataFrame with 'x', 'y', 'z' columns
df = sbj_1_session_3.electrodes.to_dataframe()

# Convert DataFrame to dictionary with electrode names as keys and 'x', 'y', 'z' coordinates as values
electrode_dict = {str(name): loc for name, loc in zip(df.index, df[['x', 'y', 'z']].values)}
# Create DigMontage
montage = make_dig_montage(ch_pos=electrode_dict, coord_frame='mni_tal')

# Assuming sbj_1_session_3.acquisition['ElectricalSeries'].data returns a numpy array with shape (43200000, 94)
data = sbj_1_session_3.acquisition['ElectricalSeries'].data[:1000000,:]
data_transposed = np.transpose(data)

# Define the sampling frequency
sfreq = 500  # modify this value to your actual sampling frequency

# Create the info structure needed by MNE
info = create_info(ch_names=list(electrode_dict.keys()), sfreq=sfreq, ch_types='ecog')

# Finally, create the Raw object
raw = RawArray(data_transposed, info)

Creating RawArray with float64 data, n_channels=94, n_times=1000000
    Range : 0 ... 999999 =      0.000 ...  1999.998 secs
Ready.


In [28]:
# from mne.viz import plot_alignment, snapshot_brain_montage
# fig = plot_alignment(
#     raw.info,
#     trans="fsaverage",
#     subject="fsaverage",
#     subjects_dir=subjects_dir,
#     surfaces=["pial"],
#     coord_frame="head",
#     sensor_colors=None,
# )
# mne.viz.set_3d_view(fig, azimuth=0, elevation=70)

# xy, im = snapshot_brain_montage(fig, raw.info)

In [33]:
sleep_stamps

,start_time,stop_time,labels
id,,,
1,86.533333,206.200000,Sleep/rest
3,206.400000,326.166667,Sleep/rest
5,326.333333,446.033333,Sleep/rest
7,446.166667,565.933333,Sleep/rest
9,566.033333,687.000000,Sleep/rest
...,...,...,...
1417,85775.900000,85895.633333,Sleep/rest
1419,85895.766667,86015.466667,Sleep/rest
1421,86015.666667,86135.433333,Sleep/rest


In [31]:
from mne import Annotations

onset = clabels_orig ['start_time'].values
duration = clabels_orig ['stop_time'].values - onset
description = clabels_orig ['labels'].values

annotations = Annotations(onset, duration, description)

raw.set_annotations(annotations)

from mne.events import events_from_annotations

events, event_id = events_from_annotations(raw)

epochs = Epochs(raw, events, event_id)

/var/folders/lw/hwgr0trd2fvc2gxpptkrl8m00000gn/T/ipykernel_43460/3951054477.py:9: RuntimeWarning: Omitted 1426 annotation(s) that were outside data range.
  raw.set_annotations(annotations)
/var/folders/lw/hwgr0trd2fvc2gxpptkrl8m00000gn/T/ipykernel_43460/3951054477.py:9: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(annotations)


ModuleNotFoundError: No module named 'mne.events'